# Lesson 4: Working with Collections

In this lesson, you'll learn how to build the `MediaCatalogue` class that stores and manages multiple movies and TV series.

---

## How to Start These Lessons

If you're reading this in a text editor and need to open it in Jupyter:

```bash
cd /path/to/pyside_tutorial
python start_lessons.py
```

**To stop the Jupyter server:** Press `Ctrl+C` in the terminal.

---

## What You'll Learn

- How to store multiple objects in a list
- Using `type()` vs `isinstance()` for filtering
- Building the MediaCatalogue class
- Adding, deleting, and filtering items
- List comprehensions for filtering

---

## Setup: The Classes We've Built

First, let's bring in the Movie and TVSeries classes from previous lessons:

In [ ]:
# Our classes from previous lessons (simplified - without full validation)
class MediaError(Exception):
    def __init__(self, message, obj):
        super().__init__(message)
        self.obj = obj

class Movie:
    def __init__(self, title, year, director, duration):
        self.title = title
        self.year = year
        self.director = director
        self.duration = duration
    
    def __str__(self):
        return f"{self.title} ({self.year}) - {self.duration} min, {self.director}"

class TVSeries(Movie):
    def __init__(self, title, year, director, duration, seasons, total_episodes):
        super().__init__(title, year, director, duration)
        self.seasons = seasons
        self.total_episodes = total_episodes
    
    def __str__(self):
        return f"{self.title} ({self.year}) - {self.seasons} seasons, {self.total_episodes} episodes"

---

## The Problem: Managing Multiple Items

You could just use a list to store movies:

In [ ]:
# Simple approach - just a list
movies = []
movies.append(Movie("The Matrix", 1999, "Wachowskis", 136))
movies.append(Movie("Inception", 2010, "Nolan", 148))

for movie in movies:
    print(movie)

This works, but it has problems:

- Nothing stops you from adding wrong things: `movies.append("not a movie")`
- No easy way to filter (get only movies, get only series)
- Logic is scattered - no central place to manage the collection

A `MediaCatalogue` class solves these problems.

---

## Building MediaCatalogue Step by Step

### Step 1: Basic Structure

Start with a class that holds a list:

In [ ]:
class MediaCatalogue:
    def __init__(self):
        self.items = []  # Empty list to store movies and series

catalogue = MediaCatalogue()
print(f"Items: {catalogue.items}")
print(f"Number of items: {len(catalogue.items)}")

### Step 2: Adding Items

Add an `add()` method that only accepts Movie or TVSeries:

In [ ]:
class MediaCatalogue:
    def __init__(self):
        self.items = []
    
    def add(self, media_item):
        """Add a media item to the catalogue."""
        # Check that it's a Movie (or TVSeries, which is a type of Movie)
        if not isinstance(media_item, Movie):
            raise MediaError("Only Movie or TVSeries can be added", media_item)
        
        self.items.append(media_item)
        return media_item  # Return the item that was added

# Test it
catalogue = MediaCatalogue()
catalogue.add(Movie("The Matrix", 1999, "Wachowskis", 136))
catalogue.add(TVSeries("Breaking Bad", 2008, "Gilligan", 47, 5, 62))

print(f"Added {len(catalogue.items)} items")
for item in catalogue.items:
    print(f"  - {item}")

In [ ]:
# Try to add something invalid
try:
    catalogue.add("not a movie")
except MediaError as e:
    print(f"Error: {e}")
    print(f"Rejected: {e.obj}")

**Why `isinstance(media_item, Movie)`?**

Remember from Lesson 2: `TVSeries` inherits from `Movie`, so a TVSeries IS a Movie.

`isinstance(tvseries, Movie)` returns `True`, so both movies and series can be added.

### Step 3: Deleting Items

Add a `delete()` method that removes an item:

In [ ]:
class MediaCatalogue:
    def __init__(self):
        self.items = []
    
    def add(self, media_item):
        if not isinstance(media_item, Movie):
            raise MediaError("Only Movie or TVSeries can be added", media_item)
        self.items.append(media_item)
        return media_item
    
    def delete(self, media_item):
        """Delete a media item from the catalogue."""
        if media_item not in self.items:
            raise MediaError("Item not found in catalogue", media_item)
        
        self.items.remove(media_item)
        return media_item  # Return the deleted item

# Test it
catalogue = MediaCatalogue()
matrix = catalogue.add(Movie("The Matrix", 1999, "Wachowskis", 136))
inception = catalogue.add(Movie("Inception", 2010, "Nolan", 148))

print(f"Before delete: {len(catalogue.items)} items")

deleted = catalogue.delete(matrix)
print(f"Deleted: {deleted}")
print(f"After delete: {len(catalogue.items)} items")

---

## Filtering: `type()` vs `isinstance()`

This is important! We need to filter movies from series. There are two ways to check types:

In [ ]:
movie = Movie("Inception", 2010, "Nolan", 148)
series = TVSeries("Breaking Bad", 2008, "Gilligan", 47, 5, 62)

# isinstance() checks if something IS that type (including inheritance)
print("isinstance checks:")
print(f"  movie is Movie: {isinstance(movie, Movie)}")
print(f"  series is Movie: {isinstance(series, Movie)}")  # True! TVSeries IS a Movie
print(f"  series is TVSeries: {isinstance(series, TVSeries)}")

print()

# type() checks the EXACT type (ignores inheritance)
print("type() checks:")
print(f"  type(movie) is Movie: {type(movie) is Movie}")
print(f"  type(series) is Movie: {type(series) is Movie}")  # False! It's TVSeries
print(f"  type(series) is TVSeries: {type(series) is TVSeries}")

**Key difference:**
- `isinstance(series, Movie)` → True (TVSeries inherits from Movie)
- `type(series) is Movie` → False (it's exactly TVSeries, not Movie)

For filtering, we need `type()` to get exact matches!

### Step 4: Filtering Methods

Add methods to get only movies or only series:

In [ ]:
class MediaCatalogue:
    def __init__(self):
        self.items = []
    
    def add(self, media_item):
        if not isinstance(media_item, Movie):
            raise MediaError("Only Movie or TVSeries can be added", media_item)
        self.items.append(media_item)
        return media_item
    
    def delete(self, media_item):
        if media_item not in self.items:
            raise MediaError("Item not found in catalogue", media_item)
        self.items.remove(media_item)
        return media_item
    
    def get_movies(self):
        """Return only Movie instances (not TVSeries)."""
        return [item for item in self.items if type(item) is Movie]
    
    def get_tv_series(self):
        """Return only TVSeries instances."""
        return [item for item in self.items if type(item) is TVSeries]

# Test it
catalogue = MediaCatalogue()
catalogue.add(Movie("The Matrix", 1999, "Wachowskis", 136))
catalogue.add(Movie("Inception", 2010, "Nolan", 148))
catalogue.add(TVSeries("Breaking Bad", 2008, "Gilligan", 47, 5, 62))
catalogue.add(TVSeries("Scrubs", 2001, "Lawrence", 24, 9, 182))

print(f"All items: {len(catalogue.items)}")
print(f"Movies only: {len(catalogue.get_movies())}")
print(f"Series only: {len(catalogue.get_tv_series())}")

print("\nMovies:")
for movie in catalogue.get_movies():
    print(f"  {movie}")

print("\nTV Series:")
for series in catalogue.get_tv_series():
    print(f"  {series}")

---

## List Comprehensions

Let's break down the list comprehension used in `get_movies()`:

```python
[item for item in self.items if type(item) is Movie]
```

In [ ]:
# A list comprehension is a compact way to filter/transform a list

# Without list comprehension:
def get_movies_long(items):
    result = []
    for item in items:
        if type(item) is Movie:
            result.append(item)
    return result

# With list comprehension (same result, shorter code):
def get_movies_short(items):
    return [item for item in items if type(item) is Movie]

# They do the same thing!
items = [Movie("Test", 2000, "Dir", 90), TVSeries("Show", 2000, "Dir", 30, 1, 10)]
print(f"Long version: {len(get_movies_long(items))} movies")
print(f"Short version: {len(get_movies_short(items))} movies")

The pattern is:
```python
[expression for item in collection if condition]
```

Read it as: "Give me `expression` for each `item` in `collection` where `condition` is true."

---

## The Complete MediaCatalogue

Here's the actual class from our project (`media_catalogue/src/media_catalogue.py` lines 38-89):

In [ ]:
class MediaCatalogue:
    """Manage a collection of media items."""
    
    def __init__(self):
        self.items = []

    def add(self, media_item):
        """Add a media item to the catalogue."""
        if not isinstance(media_item, Movie):
            raise MediaError('Only Movie or TVSeries instances can be added', media_item)
        self.items.append(media_item)
        return media_item

    def delete(self, media_item):
        """Delete a media item from the catalogue."""
        if media_item not in self.items:
            raise MediaError('Item not found in catalogue', media_item)
        self.items.remove(media_item)
        return media_item

    def get_movies(self):
        """Return a list of just the Movie instances in the catalogue."""
        return [item for item in self.items if type(item) is Movie]

    def get_tv_series(self):
        """Return a list of just the TVSeries instances in the catalogue."""
        return [item for item in self.items if type(item) is TVSeries]
    
    def filter_by_series_or_movie(self, media_type):
        """Return a list of just the Movie or TVSeries instances in the catalogue."""
        if media_type == 'movie':
            return self.get_movies()
        elif media_type == 'series':
            return self.get_tv_series()
        else:
            raise ValueError('Invalid type. Must be "movie" or "series"')
    
    def __str__(self):
        if not self.items:
            return "Media Catalogue (empty)"
        
        movies = self.get_movies()
        series = self.get_tv_series()
        result = f'Media Catalogue ({len(self.items)} items):\n\n'

        if movies:
            result += "=== MOVIES ===\n"
            for i, movie in enumerate(movies, 1):
                result += f"{i}. {movie}\n"
        
        if series:
            result += "\n=== TV SERIES ===\n"
            for i, s in enumerate(series, 1):
                result += f"{i}. {s}\n"
        
        return result

In [ ]:
# Test the complete catalogue
catalogue = MediaCatalogue()

# Add some movies
catalogue.add(Movie("The Matrix", 1999, "The Wachowskis", 136))
catalogue.add(Movie("Inception", 2010, "Christopher Nolan", 148))

# Add some series
catalogue.add(TVSeries("Breaking Bad", 2008, "Vince Gilligan", 47, 5, 62))
catalogue.add(TVSeries("Scrubs", 2001, "Bill Lawrence", 24, 9, 182))

# Print the whole catalogue
print(catalogue)

In [ ]:
# Use the filter method
print("Filtering by 'movie':")
for movie in catalogue.filter_by_series_or_movie('movie'):
    print(f"  {movie}")

print("\nFiltering by 'series':")
for series in catalogue.filter_by_series_or_movie('series'):
    print(f"  {series}")

---

## Try It Yourself

### Exercise 1: Use the Catalogue

Add 3 movies and 2 TV series to the catalogue, then print it:

In [ ]:
# Exercise 1: Add items and print the catalogue
my_catalogue = MediaCatalogue()

# Add your items here:


# Print the catalogue
print(my_catalogue)

### Exercise 2: Write a List Comprehension

Write a list comprehension that returns only movies from 2000 or later:

In [ ]:
# Exercise 2: Filter by year
test_catalogue = MediaCatalogue()
test_catalogue.add(Movie("Old Movie", 1995, "Dir", 90))
test_catalogue.add(Movie("New Movie", 2010, "Dir", 90))
test_catalogue.add(Movie("Newer Movie", 2020, "Dir", 90))

# Write a list comprehension to get movies from 2000 or later
# Hint: [item for item in test_catalogue.items if item.year >= ???]
recent_movies = [item for item in test_catalogue.items if item.year >= 2000]

print(f"Found {len(recent_movies)} recent movies")
for movie in recent_movies:
    print(f"  {movie}")

<details>
<summary><b>Show Answer</b></summary>

```python
recent_movies = [item for item in test_catalogue.items if item.year >= 2000]
```

</details>

### Exercise 3: Add a Method

Add a `count()` method to MediaCatalogue that returns the total number of items:

In [ ]:
# Exercise 3: Add a count method
# Hint: Use len(self.items)

class MediaCatalogueWithCount(MediaCatalogue):
    def count(self):
        # TODO: Return the number of items
        return len(self.items)

# Test it
cat = MediaCatalogueWithCount()
cat.add(Movie("Test 1", 2000, "Dir", 90))
cat.add(Movie("Test 2", 2000, "Dir", 90))
print(f"Count: {cat.count()}")  # Should print: Count: 2

<details>
<summary><b>Show Answer</b></summary>

```python
def count(self):
    return len(self.items)
```

</details>

---

## Key Takeaways

1. **Collections store objects** - Use a list inside your class to hold multiple items

2. **Validate on add** - Check that only valid items are added to the collection

3. **`type()` vs `isinstance()`**:
   - `isinstance()` - Checks type including inheritance (good for validation)
   - `type()` - Checks exact type only (good for filtering)

4. **List comprehensions** - Compact way to filter: `[item for item in items if condition]`

5. **Return the item** - Methods like `add()` and `delete()` return the item for convenience

---

## Summary: What We've Built So Far

In Lessons 1-4, we built the complete business logic for our Media Catalogue:

```python
# Lesson 3: Custom exception
class MediaError(Exception): ...

# Lesson 1 & 3: Movie with validation
class Movie: ...

# Lesson 2 & 3: TVSeries inherits from Movie
class TVSeries(Movie): ...

# Lesson 4: Collection to manage them all
class MediaCatalogue: ...
```

This is the **business logic** - the core functionality that doesn't depend on any user interface.

Next, we'll learn about debugging and testing before building the GUI.

---

## Next Lesson

You'll learn:
- How to use print statements effectively
- Debug flags to turn debugging on and off
- Finding and fixing bugs
- Why we'll eventually want something better (tests!)

---

## Navigation

| | |
|:---|---:|
| [**← Previous: Lesson 3 - Validation**](03_validation.ipynb) | [**Next: Lesson 5 - Debugging →**](05_debugging.ipynb) |